In [49]:
"""
Algorithm 1 LEO-Based EC Algorithm
1: Begin
2: // Initialization
3: g←0 ; // the generation index
4: Initialize population Xg and evaluate the fitness;
5: Initialize the weights of ANN randomly;
6: Initialize arch as an empty set; // to store solution pairs
7: While stop criteria not satisfied Do
8:		g←g+ 1;
9:		// Individual Evolution
10:		Sample r uniformly on [0,1];
11:		If g> 1 and r< lp Then
12:			newX ← Evolve Xg by learning-aided evolutionary operator;
13:		Else
14:			// use operators in traditional EC, e.g., PSO or DE
15:			newX ← Evolve Xg by traditional evolutionary operator;
16:		End If
17:		Evaluate the fitness of individuals in newX;
18:		// Selection
19:		Xg+1← selection among Xg and newX;
20:		// SEP Collection
21:		For each individual i in Xg+1 Do
22:			If Xg+1,i is better than Xg,i Then
23:				Add (Xg,i , Xg+1,i ) in arch;
24:			End If
25:		End For
26:		If number of SEPs in arch > arch_size Then
27:			arch ← the newest arch_size solution pairs;
28:		End If
29:		// Learning System Update
30:		Train the ANN with all data in arch for one epoch;
31:	End While
32:End
"""

'\nAlgorithm 1 LEO-Based EC Algorithm\n1: Begin\n2: // Initialization\n3: g←0 ; // the generation index\n4: Initialize population Xg and evaluate the fitness;\n5: Initialize the weights of ANN randomly;\n6: Initialize arch as an empty set; // to store solution pairs\n7: While stop criteria not satisfied Do\n8:\t\tg←g+ 1;\n9:\t\t// Individual Evolution\n10:\t\tSample r uniformly on [0,1];\n11:\t\tIf g> 1 and r< lp Then\n12:\t\t\tnewX ← Evolve Xg by learning-aided evolutionary operator;\n13:\t\tElse\n14:\t\t\t// use operators in traditional EC, e.g., PSO or DE\n15:\t\t\tnewX ← Evolve Xg by traditional evolutionary operator;\n16:\t\tEnd If\n17:\t\tEvaluate the fitness of individuals in newX;\n18:\t\t// Selection\n19:\t\tXg+1← selection among Xg and newX;\n20:\t\t// SEP Collection\n21:\t\tFor each individual i in Xg+1 Do\n22:\t\t\tIf Xg+1,i is better than Xg,i Then\n23:\t\t\t\tAdd (Xg,i , Xg+1,i ) in arch;\n24:\t\t\tEnd If\n25:\t\tEnd For\n26:\t\tIf number of SEPs in arch > arch_size Then\

In [50]:
from pymoo.optimize import minimize
import numpy as np
import random
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.core.population import Population
from pymoo.core.evaluator import Evaluator
from keras.layers import Dense
from keras.models import Sequential

In [51]:
def normalize(data):
    min_val = -100
    max_val = 100
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data


def denormalize(data):
    min_val = -100
    max_val = 100
    denormalized_data = (data * (max_val - min_val)) + min_val
    return denormalized_data

In [52]:
def generate_model(n_var):
    model = Sequential()
    model.add(Dense((3*n_var), input_dim=n_var, activation='sigmoid'))
    model.add(Dense(n_var, activation='sigmoid'))

    model.compile(loss='mse',
                  optimizer='adam', metrics=['accuracy'])
    return model

In [53]:
def traditional_de(problem, pop_size=100, generation=1000, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    X = np.random.uniform(l_bound, h_bound, (pop_size, n_var))
    pop = Population.new("X", X)
    Evaluator().eval(problem, pop)

    algorithm = DE(
        pop_size=pop_size,
        sampling=pop,
        variant="DE/rand/1/bin",
        CR=0.3,
        dither="vector",
        jitter=False)

    res = minimize(problem,
                   algorithm,
                   ("n_gen", generation),
                   verbose=verbose)
    return res.pop.get("X"), res.pop.get("F"), res.X, res.F

In [54]:

from pymoo.core.problem import Problem


class Benchmark(Problem):
    def __init__(self, func, n_var=2):
        super().__init__(n_var=n_var, n_obj=1, n_ieq_constr=0, xl=-100, xu=100, vtype=float)
        self.f = func(self.n_var, bounds=(self.xl, self.xu))

    def _evaluate(self, x, out):
        F = []
        for i in x:
            F.append(self.f.evaluate(i))
        out["F"] = np.array(F)

In [55]:
from opfunu.cec_based.cec2021 import F12021, F22021, F32021, F42021, F52021, F72021, F82021, F92021, F102021


n_var = 10

problem = Benchmark(func=F52021, n_var=n_var)

pop_traditional, fit_traditional, x_best_traditional, f_best_traditional = traditional_de(problem, pop_size=100, generation=1000,
                                                                                          n_var=n_var, l_bound=-100, h_bound=100, verbose=True)

F52021 problem is set with 10 variables!
n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |        0 |  3.999661E+09 |  1.263865E+07
     2 |      100 |  3.070669E+09 |  4.421200E+06
     3 |      200 |  2.235859E+09 |  4.362487E+06
     4 |      300 |  1.984722E+09 |  4.043775E+06
     5 |      400 |  1.315139E+09 |  4.043775E+06
     6 |      500 |  9.205979E+08 |  3.548503E+05
     7 |      600 |  8.480857E+08 |  3.548503E+05
     8 |      700 |  5.750767E+08 |  3.548503E+05
     9 |      800 |  4.480778E+08 |  3.548503E+05
    10 |      900 |  3.535128E+08 |  3.548503E+05
    11 |     1000 |  2.831953E+08 |  3.548503E+05
    12 |     1100 |  2.111428E+08 |  3.548503E+05
    13 |     1200 |  1.788603E+08 |  2.079148E+05
    14 |     1300 |  1.624951E+08 |  2.079148E+05
    15 |     1400 |  1.143671E+08 |  2.079148E+05
    16 |     1500 |  9.889015E+07 |  1.639951E+05
    17 |     1600 |  7.751769E+07 |  1.639951E+05
    18 |     1700 |  7.451194E+07 |  1.639951E+05
    19 | 

In [56]:

def leo_based_de(problem, pop_size=100, generation=100, lp=0.1, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    pop = np.random.uniform(l_bound, h_bound, (pop_size, problem.n_var))

    # pop = Population.new("X", X)
    fit = problem.evaluate(pop)
    # Evaluator().eval(problem, pop)

    model = generate_model(n_var)

    archive_x = []
    archive_y = []

    for g in range(generation):
        if g > 0 and random.random() < lp and len(archive_x) > 0:
            pred = model.predict(normalize(pop), verbose=verbose)
            pop_new = denormalize(pred)
            fit_new = problem.evaluate(pop_new)
        else:
            algorithm = DE(
                pop_size=pop_size,
                sampling=pop,
                variant="DE/rand/1/bin",
                CR=0.5,
                dither="vector",
                jitter=False)
            res = minimize(problem, algorithm, ("n_gen", 2), verbose=False)
            pop_new = res.pop.get("X")
            fit_new = res.pop.get("F")
        for i in range(fit_new.shape[0]):
            if fit_new[i] < fit[i]:
                archive_x.append(pop[i])
                archive_y.append(pop_new[i])
        pop = pop_new
        fit = fit_new
        if len(archive_x) > 0:
            archive_x = archive_x[-100:]
            archive_y = archive_y[-100:]
            model.fit(normalize(np.array(archive_x)), normalize(
                np.array(archive_y)), epochs=1, verbose=verbose)
        print(f"gen:  {g}, archive size: {len(archive_x)}", end='       \r')
    return pop, fit, res.X, res.F

In [57]:
# n_var = 10
# problem = Benchmark(func=F12021, n_var=n_var)
pop_leo, fit_leo, x_best_leo, f_best_leo = leo_based_de(problem, pop_size=100, generation=1000, lp=0.1,
                                                        n_var=n_var, l_bound=-100, h_bound=100, verbose=False)

In [58]:
from scipy.stats import friedmanchisquare, wilcoxon

In [59]:
statistic, pvalue = wilcoxon(fit_leo)
pvalue

array([3.89655985e-18])

In [60]:
statistic, pvalue = wilcoxon(fit_traditional)
pvalue

array([3.89655985e-18])